# 处理附件3中数据

In [ ]:
import pandas as pd
import numpy as np
import os
from pprint import pprint
from scipy import interpolate

# 1 加载数据

In [14]:
A1H_pre_raw = pd.read_excel('./data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx',sheet_name='监测点A1逐小时污染物浓度与气象一次预报数据')
A2H_pre_raw = pd.read_excel('./data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx',sheet_name='监测点A2逐小时污染物浓度与气象一次预报数据')
A3H_pre_raw = pd.read_excel('./data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx',sheet_name='监测点A3逐小时污染物浓度与气象一次预报数据')

A1H_act_raw = pd.read_excel('./data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx',sheet_name='监测点A1逐小时污染物浓度与气象实测数据')
A2H_act_raw = pd.read_excel('./data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx',sheet_name='监测点A2逐小时污染物浓度与气象实测数据')
A3H_act_raw = pd.read_excel('./data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx',sheet_name='监测点A3逐小时污染物浓度与气象实测数据')

A1D_act_raw = pd.read_excel('./data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx',sheet_name='监测点A1逐日污染物浓度实测数据')
A2D_act_raw = pd.read_excel('./data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx',sheet_name='监测点A2逐日污染物浓度实测数据')
A3D_act_raw = pd.read_excel('./data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx',sheet_name='监测点A3逐日污染物浓度实测数据')

In [15]:
A1H_pre = A1H_pre_raw
A2H_pre = A2H_pre_raw
A3H_pre = A3H_pre_raw

A1H_act = A1H_act_raw
A2H_act = A2H_act_raw
A3H_act = A3H_act_raw

A1D_act = A1D_act_raw
A2D_act = A2D_act_raw
A3D_act = A3D_act_raw

In [5]:
A1H_act.columns

Index(['监测时间', '地点', 'SO2监测浓度(μg/m³)', 'NO2监测浓度(μg/m³)', 'PM10监测浓度(μg/m³)',
       'PM2.5监测浓度(μg/m³)', 'O3监测浓度(μg/m³)', 'CO监测浓度(mg/m³)', '温度(℃)', '湿度(%)',
       '近地风速(m/s)', '风向(°)'],
      dtype='object')

In [6]:
A1H_pre.set_axis(['start_time','time', 'place', 'temperature_2m','temperature_surface','specific_humidity','humidity','wind_speed_10m','wind_direction_10m','rainfall','cloudiness','height','pressure','sensible_heat','latent_heat','longwave','shortwave','solar_radiation','so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)
A1H_act.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co','temperature', 'humidity','wind','direction'], axis='columns', inplace=True)

A2H_pre.set_axis(['start_time','time', 'place', 'temperature_2m','temperature_surface','specific_humidity','humidity','wind_speed_10m','wind_direction_10m','rainfall','cloudiness','height','pressure','sensible_heat','latent_heat','longwave','shortwave','solar_radiation','so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)
A2H_act.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co','temperature', 'humidity','pressure','wind','direction'], axis='columns', inplace=True)

A3H_pre.set_axis(['start_time','time', 'place', 'temperature_2m','temperature_surface','specific_humidity','humidity','wind_speed_10m','wind_direction_10m','rainfall','cloudiness','height','pressure','sensible_heat','latent_heat','longwave','shortwave','solar_radiation','so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)
A3H_act.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co','temperature', 'humidity','pressure','wind','direction'], axis='columns', inplace=True)


A1D_act.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)
A2D_act.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)
A3D_act.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)

A1D_act = A1D_act[0:-3]
A2D_act = A2D_act[0:-3]
A2D_act = A3D_act[0:-3]


In [7]:
# 暂时删除字符串的列，方便后续插值处理，后续会添加回去的
A1H_pre_time = A1H_pre['time']
A2H_pre_time  = A2H_pre['time']
A3H_pre_time  = A3H_pre['time']

A1H_act_time  = A1H_act['time']
A2H_act_time  = A2H_act['time']
A3H_act_time  = A3H_act['time']

A1D_act_time  = A1D_act['time']
A2D_act_time  = A2D_act['time']
A3D_act_time  = A3D_act['time']
#C_hour_actual表的humidity这一列缺失值太多，直接舍弃

A1H_pre = A1H_pre.drop(columns=['place','time','start_time'])
A2H_pre = A2H_pre.drop(columns=['place','time','start_time'])
A3H_pre = A3H_pre.drop(columns=['place','time','start_time'])

A1H_act = A1H_act.drop(columns=['place','time'])
A2H_act = A2H_act.drop(columns=['place','time'])
A3H_act = A3H_act.drop(columns=['place','time'])

A1D_act = A1D_act.drop(columns=['place','time'])
A2D_act = A2D_act.drop(columns=['place','time'])
A3D_act = A3D_act.drop(columns=['place','time'])

In [8]:
# 缺失值统一化
A1H_pre = A1H_pre.replace('—', np.nan).replace('NULL', np.nan).astype('float')
A2H_pre = A2H_pre.replace('—', np.nan).replace('NULL', np.nan).astype('float')  
A3H_pre = A3H_pre.replace('—', np.nan).replace('NULL', np.nan).astype('float')  

A1H_act = A1H_act.replace('—', np.nan).replace('NULL', np.nan).astype('float')  
A2H_act = A2H_act.replace('—', np.nan).replace('NULL', np.nan).astype('float')  
A3H_act = A3H_act.replace('—', np.nan).replace('NULL', np.nan).astype('float')  

A1D_act = A1D_act.replace('—', np.nan).replace('NULL', np.nan).astype('float')  
A2D_act = A2D_act.replace('—', np.nan).replace('NULL', np.nan).astype('float')  
A3D_act = A3D_act.replace('—', np.nan).replace('NULL', np.nan).astype('float')  

In [9]:

# 异常值处理
A1H_pre[ A1H_pre< 0 ] = np.nan
A2H_pre[ A2H_pre< 0 ] = np.nan
A3H_pre[ A3H_pre< 0 ] = np.nan

A1H_act[ A1H_act< 0 ] = np.nan
A2H_act[ A2H_act< 0 ] = np.nan
A3H_act[ A3H_act< 0 ] = np.nan

A1D_act[ A1D_act< 0 ] = np.nan
A2D_act[ A2D_act< 0 ] = np.nan
A3D_act[ A3D_act< 0 ] = np.nan


A1H_pre.insert(0, 'time', A1H_pre_time)
A2H_pre.insert(0, 'time', A2H_pre_time)
A3H_pre.insert(0, 'time', A3H_pre_time)

A1H_act.insert(0, 'time', A1H_act_time)
A2H_act.insert(0, 'time', A2H_act_time)
A3H_act.insert(0, 'time', A3H_act_time)

A1D_act.insert(0, 'time', A1D_act_time)
A2D_act.insert(0, 'time', A2D_act_time)
A3D_act.insert(0, 'time', A3D_act_time)

In [10]:
A1D_act.columns

Index(['time', 'so2', 'no2', 'pm10', 'pm2.5', 'o3', 'co'], dtype='object')

# 2 插值

## 2.1 线性插值

In [ ]:

filedict = {"A1HP":A1H_pre,"A2HP":A2H_pre,"A3HP":A3H_pre,"A1HA":A1H_act,"A2HA":A2H_act,"A3HA":A3H_act,"A1DA":A1D_act,"A2DA":A2D_act,"A3DA":A3D_act}
for k,data in filedict.items():
    for indexs in data.columns:
        if indexs =='time':
            continue
        data['rownum'] = np.arange(data.shape[0])
        df_nona = data.dropna(subset = [indexs])
        f = interpolate.interp1d(df_nona['rownum'], df_nona[indexs])
        data[indexs] = f(data['rownum'])

    data = data.drop(columns=['rownum'])
    # 把地点加回去
    data['place'] = [k[0]]*len(data)
    if k =="A1HP":
        data_A1HP = data
    elif k =="A2HP":
        data_A2HP = data
    elif k =="A3HP":
        data_A3HP = data
    elif k =="A1HA":
        data_A1HA = data
    elif k =="A2HA":
        data_A2HA = data
    elif k =="A3HA":
        data_A3HA = data
    elif k =="A1DA":
        data_A1DA = data
    elif k =="A2DA":
        data_A2DA = data
    elif k =="A3DA":
        data_A3DA = data

In [ ]:
data_BHA

In [ ]:
writer=pd.ExcelWriter('data/data_3_linear.xlsx')   #定义writer
data_A1HP.to_excel(writer,'监测点A1逐小时污染物浓度与气象一次预报数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_A2HP.to_excel(writer,'监测点A2逐小时污染物浓度与气象一次预报数据',index=False)
data_A3HP.to_excel(writer,'监测点A3逐小时污染物浓度与气象一次预报数据',index=False) # writer---writer；sheet名称---原始表（35698）

data_A1HA.to_excel(writer,'监测点A1逐小时污染物浓度与气象实测数据',index=False)
data_A2HA.to_excel(writer,'监测点A2逐小时污染物浓度与气象实测数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_A3HA.to_excel(writer,'监测点A3逐小时污染物浓度与气象实测数据',index=False)

data_A1DA.to_excel(writer,'监测点A1逐日污染物浓度实测数据',index=False)
data_A2DA.to_excel(writer,'监测点A2逐日污染物浓度实测数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_A3DA.to_excel(writer,'监测点A3逐日污染物浓度实测数据',index=False)
writer.save()

## 2.2 KNN插值

In [11]:
filedict = {"A1HP":A1H_pre,"A2HP":A2H_pre,"A3HP":A3H_pre,"A1HA":A1H_act,"A2HA":A2H_act,"A3HA":A3H_act,"A1DA":A1D_act,"A2DA":A2D_act,"A3DA":A3D_act}

for k,data in filedict.items():
    def knn_mean(ts, n):
        out = np.copy(ts)
        for i, val in enumerate(ts):
            if np.isnan(val):
                n_by_2 = np.ceil(n/2)
                lower = np.max([0, int(i-n_by_2)])
                upper = np.min([len(ts)+1, int(i+n_by_2)])
                ts_near = np.concatenate([ts[lower:i], ts[i:upper]])
                out[i] = np.nanmean(ts_near)
        return out
    for indexs in data.columns:
        if indexs =='time':
            continue
        data[indexs] = knn_mean(data[indexs].values,24)
    if k =="A1HP":
        data_A1HP = data
    elif k =="A2HP":
        data_A2HP = data
    elif k =="A3HP":
        data_A3HP = data
    elif k =="A1HA":
        data_A1HA = data
    elif k =="A2HA":
        data_A2HA = data
    elif k =="A3HA":
        data_A3HA = data
    elif k =="A1DA":
        data_A1DA = data
    elif k =="A2DA":
        data_A2DA = data
    elif k =="A3DA":
        data_A3DA = data

<ipython-input-11-e459cdbf532b>:12: RuntimeWarning: Mean of empty slice
  out[i] = np.nanmean(ts_near)


In [12]:
writer=pd.ExcelWriter('data/data_3_knn.xlsx')   #定义writer
data_A1HP.to_excel(writer,'监测点A1逐小时污染物浓度与气象一次预报数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_A2HP.to_excel(writer,'监测点A2逐小时污染物浓度与气象一次预报数据',index=False)
data_A3HP.to_excel(writer,'监测点A3逐小时污染物浓度与气象一次预报数据',index=False) # writer---writer；sheet名称---原始表（35698）

data_A1HA.to_excel(writer,'监测点A1逐小时污染物浓度与气象实测数据',index=False)
data_A2HA.to_excel(writer,'监测点A2逐小时污染物浓度与气象实测数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_A3HA.to_excel(writer,'监测点A3逐小时污染物浓度与气象实测数据',index=False)

data_A1DA.to_excel(writer,'监测点A1逐日污染物浓度实测数据',index=False)
data_A2DA.to_excel(writer,'监测点A2逐日污染物浓度实测数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_A3DA.to_excel(writer,'监测点A3逐日污染物浓度实测数据',index=False)
writer.save()